In [ ]:
!pip install langchain-core langgraph>0.2.27

In [ ]:
!pip install -U langchain-community

  Using cached langchain-0.3.27-py3-none-any.whl.metadata (7.8 kB)
Using cached langchain-0.3.27-py3-none-any.whl (1.0 MB)


In [ ]:
!pip install "langchain[google-genai]" google-ai-generativelanguage==0.6.15

  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
  Using cached google_generativeai-0.8.5-py3-none-any.whl.metadata (3.9 kB)
Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl (1.3 MB)
Using cached google_generativeai-0.8.5-py3-none-any.whl (155 kB)


In [ ]:
!pip install pypdf chromadb

In [ ]:
import os

In [ ]:
os.environ["GOOGLE_API_KEY"] ='YOUR_API_KEY'

In [ ]:
import zipfile

zip_path = "/content/Topics.zip"
extract_to = "/content/extracted_topics" # Corrected extraction path

# Unzip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader("/content/extracted_topics/Topics/p70-178.pdf")


documents = loader.load()


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)


chunks = text_splitter.split_documents(documents)

In [ ]:
import zipfile
import os

zip_file_path = "/content/Topics.zip"

extract_dir = "/content/extracted_topics"

os.makedirs(extract_dir, exist_ok=True)


with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Zip file '{zip_file_path}' extracted to '{extract_dir}'.")
pdf_file_path = None
for root, _, files in os.walk(extract_dir):
    for file in files:
        if file.endswith(".pdf"):
            pdf_file_path = os.path.join(root, file)
            break
    if pdf_file_path:
        break

if pdf_file_path:
    print(f"Found PDF file: {pdf_file_path}")
    from langchain_community.document_loaders import PyPDFLoader
    from langchain.text_splitter import RecursiveCharacterTextSplitter

    loader = PyPDFLoader(pdf_file_path)


    documents = loader.load()


    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

    chunks = text_splitter.split_documents(documents)

    print(f"Number of chunks: {len(chunks)}")
else:
    print("No PDF file found in the extracted directory.")


Found PDF file: /content/extracted_topics/Topics/p70-178.pdf
Number of chunks: 110


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

embeddings = embeddings_model.embed_documents([chunk.page_content for chunk in chunks])

print(f"Generated embeddings for {len(embeddings)} chunks.")

Generated embeddings for 110 chunks.


In [ ]:
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings_model)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")


llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")


document_chain = create_stuff_documents_chain(llm, prompt)


retrieval_chain = create_retrieval_chain(vectorstore.as_retriever(), document_chain)

In [ ]:
print("Starting RAG system testing...")

sample_questions = [
    "What is the topic of the document?",
    "Can you summarize the main points?",
    "Are there any dates mentioned?",
    "What is the significance of the numbers mentioned in the document?"
]

for question in sample_questions:
    print(f"\nQuestion: {question}")
    response = retrieval_chain.invoke({"input": question})
    print(f"Answer: {response['answer']}")

print("\nFinished RAG system testing.")

Starting RAG system testing...

Question: What is the topic of the document?
Answer: The document's topic is occupation, earnings, and job characteristics among the U.S. population.  It uses data from the 2018 Survey of Income and Program Participation (SIPP) and the 2018 American Community Survey (ACS) to describe common employment features, including occupations, work schedules, earnings, and other job characteristics.

Question: Can you summarize the main points?
Answer: The provided text discusses job quality, focusing on earnings and employee benefits.  It highlights that monetary compensation (wages, salaries, tips, etc.) is a key factor impacting standard of living.  The data shows variations in median monthly income across different occupations, with computer and mathematical occupations, and architecture and engineering occupations having the highest, and personal care and service occupations having the lowest.  The report also notes differences in the rates of part-time work 